# Data import

To connect and get a terminal to the postgres docker, run `docker exec -it pbdw2018_hackathon-master_data_postgres_1 /bin/bash`

You can then do things like

dropdb mydata -U postgres

or

psql postgres -U postgres


In [14]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import csv

# Connect to an existing database
try:
    conn = psycopg2.connect("host=postgresdb user=postgres password=postgres")
except:
    print ("Error:  unable to connect to the database")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command to end all connections to the db
try:
    cur.execute("SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = 'mydata' AND pid <> pg_backend_pid();")
except:
    print("Error killing database connections, perhaps it does not exist?")
    
# Execute a command to drop the table
try:
    cur.execute("DROP DATABASE mydata")
except:
    print("Error while dropping database, perhaps it does not exist?")

# Execute a command to create a new table
try:
    cur.execute("CREATE DATABASE mydata;")
except:
    print("Error while creating database, does it already exist?")
# Close database connection
conn.close()

In [3]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a command: this creates a new table
try:
    cur.execute("""CREATE TABLE PBDWHackathon2018 (
    ID text PRIMARY KEY,
    PatientID text,
    Age text,
    Clinical_T_Stage text,
    Clinical_N_Stage text,
    Clinical_M_Stage text,
    Overall_Stage text,
    Histology text,
    Gender text,
    SurvivalTime text,
    DeadStatus text
    );
    """)
except:
    print("Error while creating table, does it already exist?")
# Close database connection
conn.close()

In [4]:
import pandas
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

#open CSV file
df = pandas.read_csv('Clinical1.csv')
for index, row in df.iterrows():
    #print(str(index) + " | " + row.Idfu)
    #cur.execute
    cur.execute("""INSERT INTO PBDWHackathon2018 (ID, PatientID, Age, Clinical_T_Stage, Clinical_N_Stage, Clinical_M_Stage, Overall_Stage, Histology, Gender, SurvivalTime, DeadStatus) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
          (index, row.PatientID, row.Age, row["Clinical.T.Stage"], row["Clinical.N.Stage"], row["Clinical.M.Stage"], row["Overall.Ajcc.Stage"], row.Histology, row.Sex, row["Survival.Time.Days"], row["deadstatus.event"]))
conn.commit()

In [5]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

cur.execute("""SELECT *
    FROM PBDWHackathon2018;""")

results = cur.fetchall()
conn.close()

for row in results:
    print(row)

('0', 'LUNG1-001', '79', '2', '3', '0', 'IIIb', 'large_cell', 'male', '2165', '1')
('1', 'LUNG1-002', '84', '2', '0', '0', 'I', 'scc', 'male', '155', '1')
('2', 'LUNG1-003', '68', '2', '3', '0', 'IIIb', 'large_cell', 'male', '256', '1')
('3', 'LUNG1-004', '71', '2', '1', '0', 'II', 'scc', 'male', '141', '1')
('4', 'LUNG1-005', '80', '4', '2', '0', 'IIIb', 'scc', 'male', '353', '1')
('5', 'LUNG1-006', '74', '3', '1', '0', 'IIIa', 'scc', 'male', '173', '1')
('6', 'LUNG1-007', '82', '2', '2', '0', 'IIIa', 'scc', 'male', '137', '1')
('7', 'LUNG1-008', '72', '2', '2', '0', 'IIIa', 'adeno', 'male', '77', '1')
('8', 'LUNG1-009', '56', '2', '2', '0', 'IIIa', 'scc', 'male', '131', '1')
('9', 'LUNG1-010', '71', '4', '3', '0', 'IIIb', 'scc', 'female', '2119', '0')


In [3]:
import pandas as pd
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [6]:
class ManageDB:
    """ Class for interaction with database
    """

    def __init__(self, dbname, host, port, user, password):
        """ Initialisation method of ManageDB class

        Parameters
        ----------
        dbname : str
            Database name
        host : str
            Host address
        port : int
            Connection port number
        user : str
            Database username
        password : str
            User's password
        """
        self.dbname = dbname
        self.host = host
        self.port = port
        self.user = user
        self.password = password

    def connect(self):
        """ Connect to database
        """
        try:
            self.conn = connect(
                dbname=self.dbname,
                host=self.host,
                port=self.port,
                user=self.user,
                password=self.password
            )
            self.conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        except Exception as e:
            raise Exception('Unable to connect to the database: {e}')

    def create_table(self, table, columns):
        """ Create a new table

        Parameters
        ----------
        table : str
            Table name
        columns : dict
            Dictionary with column names as keys and their data types as values
        """
        try:
            # Open cursor to perform database operation
            cursor = self.conn.cursor()

            # Flatten dictionary with columns to a string
            columns = ','.join(map(' '.join, columns.items()))

            # Create table
            create_table = 'CREATE TABLE %s (%s);' % (table, columns)
            cursor.execute(create_table)
        except Exception as e:
            raise Exception('Unable to create table: {e}')

    def prepare_data(self, file_path, columns, rename=None):
        """ Prepare data from CSV input

        Parameters
        ----------
        file_path : str
            Path to csv file
        columns : list
            List with relevant columns
        rename : dict
            Dictionary with mapping for column names

        Returns
        -------
        df : pd.DataFrame
            Dataframe with input data
        """

        # Read csv file
        df = pd.read_csv(file_path)

        # Rename columns when necessary
        if rename:
            df.rename(columns=rename, inplace=True)

        # Make sure that columns are in the correct order and that only
        # relevant ones are selected
        df = df[columns]

        return df

    def insert_data(self, table, columns, df):
        """ Insert new data into table

        Parameters
        ----------
        table : str
            Table name
        columns : list
            List with column names
        df : pd.DataFrame
            Dataframe with data to be inserted, we assume that the columns
            are in the correct order
        """
        try:
            # Open cursor to perform database operation
            cursor = self.conn.cursor()

            # Insertion statement
            # TODO: retrieve column names automatically?
            cols = ', '.join(columns)
            values = ', '.join(['%s']*df.shape[1])
            insert = 'INSERT INTO %s (%s) VALUES (%s)' % (table, cols, values)

            # Insert new data
            for index, row in df.iterrows():
                cursor.execute(insert, tuple(row.values))

            # Commit data insertion
            self.conn.commit()
        except Exception as e:
            raise Exception('Unable to insert new data: {e}')

    def show_table(self, table, n=10):
        """ Display top n rows of a table

        Parameters
        ----------
        table : str
            Table name
        n : int
            Number of rows to display
        """
        try:
            # Open cursor to perform database operation
            cursor = self.conn.cursor()

            # Query table
            query = 'SELECT * FROM %s;' % table
            cursor.execute(query)
            results = cursor.fetchall()

            # Display results
            for row in results[:n]:
                print(row)
        except Exception as e:
            raise Exception('Unable to display table: {e}')

    def close(self):
        """ Close database connection
        """
        self.conn.close()

In [19]:
dbname = 'mydata'
host = 'postgresdb'
port = 5432
user = 'postgres'
password = 'postgres'
table = 'CORAL'
columns = {
    'id': 'text PRIMARY KEY',
    't': 'text',
    'n': 'text',
    'm': 'text',
    'stage': 'text',
    'date_of_diagnosis': 'text',
    'date_of_fu': 'text',
    'vital_status': 'text'
}
file_path = '20k_sample_data.csv'

In [16]:
postgres = ManageDB(
    dbname=dbname, host=host, port=port, user=user, password=password
)
postgres.connect()

In [17]:
postgres.create_table(table=table, columns=columns)

In [20]:
df = postgres.prepare_data(file_path=file_path, columns=columns.keys())

In [21]:
postgres.insert_data(table=table, columns=df.columns, df=df)

In [22]:
postgres.show_table(table=table)

('T1', '2012-01-01', 'dead', '2015-01-01', 'I', 'M0', '1', 'N0')
('T2', '2013-03-05', 'dead', '2015-01-01', 'IA', 'M1', '2', 'N1')
('T3', '2014-04-16', 'alive', '2018-01-01', 'IB', 'M0', '3', 'N2')
('T4', '2015-05-06', 'alive', '2016-01-01', 'II', 'M1c', '4', 'N3')
('Tx', '2012-01-01', 'dead', '2015-01-01', 'IIA', 'Mx', '5', 'Nx')
('Tis', '2012-01-01', 'dead', '2015-01-01', 'IIB', 'M1a', '6', 'Nx')
('T1mi', '2012-01-01', 'dead', '2015-01-01', 'III', 'M1b', '7', 'N0')
('T1a', '2012-01-01', 'dead', '2015-01-01', 'IIIA', 'Mx', '8', 'N0')
('T1b', '2012-01-01', 'dead', '2015-01-01', 'IIIB', 'Mx', '9', 'N0')
('Tis', '2012-01-01', 'dead', '2015-01-01', '0', 'Mx', '10', 'N0')


In [23]:
postgres.close()